### Step 1: Set Parameters

In [1]:
import numpy as np
import pandas as pd

In [2]:
N_GENOMES = 5
GENOME_LEN = 1000

N_GENES = 20
GENE_LEN = 100
P_MUTATION = .03

### Step 2: Simulate Genomes

In [3]:
# For now, keep genomes as a string
# This will make simulating the genes more convenient due to Numpy's indexing
genomes = np.random.choice(['A', 'C', 'G', 'T'], (N_GENOMES,GENOME_LEN))

### Step 3: Simulate Genes
For each gene, save the text of the mutated gene, index of the source genome, the start index within the genome, and the number of mutations applied

In [4]:
source_g = np.random.choice(N_GENOMES, N_GENES)
start_i = np.random.choice(GENOME_LEN - GENE_LEN, N_GENES)

genes = pd.DataFrame({"source_g": source_g, "start_i": start_i})
print(genes.head())

   source_g  start_i
0         3      598
1         1      829
2         3       57
3         3      694
4         3      660


In [5]:
CHOICES = {
    'A': ['C', 'G', 'T'],
    'C': ['A', 'G', 'T'],
    'G': ['A', 'C', 'T'],
    'T': ['A', 'C', 'G'],
}

def mutate(source_g, start_i, genomes):
    gene = (genomes[source_g, start_i:start_i + GENE_LEN]).copy()
    n_mutations = np.random.binomial(GENE_LEN, P_MUTATION)
    mutation_indices = np.random.choice(GENE_LEN, n_mutations)
    for i in mutation_indices:
        gene[i] = np.random.choice(CHOICES[gene[i]])

    mutations = GENE_LEN - (genomes[source_g, start_i:start_i + GENE_LEN] == gene).sum()
    return "".join(gene), mutations
    

In [6]:
genes["raw_mutation_result"] = genes.apply(lambda row: mutate(row[0], row[1], genomes), axis=1)
genes["full_text"] = genes["raw_mutation_result"].apply(lambda x: x[0])
genes["n_mutations"] = genes["raw_mutation_result"].apply(lambda x: x[1])
genes.drop("raw_mutation_result", axis=1, inplace=True)

# Now that we're done simulating genes, join genome as a string
genomes = np.apply_along_axis(lambda row: "".join(row),1,genomes)

### Step 4: Evaluate
How often does the longest common substring align with the gene's location?

In [7]:
from difflib import SequenceMatcher

str1 = 'GeeksforGeeks'
str2 = 'GeeksQuiz'

seqMatch = SequenceMatcher(None,str1,str2)

In [8]:
match = seqMatch.find_longest_match(0, len(str1), 0, len(str2))
match

Match(a=0, b=0, size=5)

In [9]:
match.a

0

In [10]:
# return predicted start index of gene in genome
matcher = SequenceMatcher()


def predict_location(gene, source_g):
    matcher.set_seqs(gene, genomes[source_g])
    match = matcher.find_longest_match(0, GENE_LEN, 0, GENOME_LEN)
    
    return match.b - match.a

predict_location(genes.full_text[0], genes.source_g[0])

0

In [13]:
genes.head()

,source_g,start_i,full_text,n_mutations
0,3,598,GAGGATCCCGGTAAATGCAGCCATTCCCACTGCCTCGAGCTATTCT...,2
1,1,829,AAAACAAACCACTACCGGGGGAATAAAGGTGCATCGAGCTTAAGCG...,1
2,3,57,CCTGAGGGCCACCGACCTTATAGATGAATGCCAACCGTGTATATAC...,3
3,3,694,CAGTGATTCCGAGGACACCACTCAGCGTCGCAGTTAGTATTTCTCG...,0
4,3,660,TAAATTCTGACTTGTCCGGGTAAGCCACCTCGTACATTGATTCCGA...,4


In [14]:
(gene, source_g) = (genes.full_text[0], genes.source_g[0])
matcher.set_seqs(gene, genomes[source_g])
match = matcher.find_longest_match(0, GENE_LEN, 0, GENOME_LEN)

print(f"Length: {match.size}\tgenome_i: {match.b}\tgene_i: {match.a}")

Length: 0	genome_i: 0	gene_i: 0


In [16]:
matcher.a

'GAGGATCCCGGTAAATGCAGCCATTCCCACTGCCTCGAGCTATTCTGGTCTGGGACCATCAATAAATTCTGTCTTGTCCGTGTAAGCCACCTCGTACAGT'

In [17]:
matcher.b

'TTAAAGTCGGCGTGCAACAAAACTCAGAGGCCGAATATTGTTACAACTATGGTACCACCTGAGGGCCCCCGTCCTTATAGATGAATGCCAACCGTGTATATACCACACGATTGCAACGGCGCGCTTAAGCTCCAACTACTTAGTCTCTATAGAGGAGATGCTAGTGTCTAGTGTGGAGACTGCAGCCGGATCGATGGGGGCGATTGGTTGATGTCTACTGGCGCTCAAGTCGATCAGGCCTTTGGGGAGCACACAGATGTATGGTCCGGTTGATCAGAGCCTCATAAGACCCCGAATGTATTGTCCCGCATACTGCCTTTCGCCGTTCGAAAAGAACTGCGATTAGTGTTTATCGTAGCGGGTGCCGGAGGGCCTTAATTGGTTCCAGACGCTTTTTCTCAGTGATATAGAAGTTTAAGCGAAGAGGTATGTGTTCCGGGTCGTATACGGTCGGGGTTAAGAGCTTGGCACCATTTATAGCTAGCCAGTTTACGGATGCAGTCGGGCCGCGCTTATTGATACTTGCAAAACCGATCACTCGCTAATCATGTGGATAACCCATTATTGATCGGCCTATTTCTTTCAATGGGATCTGGTGGAGGATCCCGGTAAATGCAGCCAGTCCCACTGCCTCGAGCTATTCTGGTCTGGGACCATCAATAAATTCTGACTTGTCCGTGTAAGCCACCTCGTACAGTGATTCCGAGGACACCACTCAGCGTCGCAGTTAGTATTTCTCGGTGTATGACCTAGCTGTTCCTTGTCGCAACTACCAATGAGCCCCACAACCGGAGTCTTTCCCAGCTAATTCGAGCTCAACTACGCTCGAAAAGGATCACCCATTTACTGTTATACGTGCAGGAGCTACAGTGCCAATACATGCAGTAAAGAGTTTGGGGTTACGTGGGACCTCTGTCCCTCACCTAACTTTTTGGGTACTTGAGCCGCGGGGCTTGTCACGGACACCTAGGCCGAGCTCTCAATCCCAGGAGAAACAGG

In [18]:
matcher.find_longest_match(0,7,0,7)

Match(a=0, b=0, size=0)

In [56]:
#. matcher.set_seqs("GAGGATCCCGGTAAATGCAGCCATTCCCACTGCCTCGAGCTATTCTGGTCTGGGACCATCAATAAATTCTGTCTTGTCCGTGTAAGCCACCTCGTACAGT", "TTAAAGTCGGCGTGAGGATCCC")
matcher.set_seqs(str(gene), str(genomes[source_g])[200:400])
matcher.find_longest_match(0,len(matcher.a),0,len(matcher.b))

Match(a=0, b=0, size=0)

In [45]:
matcher.b

'TTAAAGTCGGCGTGCAACAAAACTCAGAGGCCGAATATTGTTACAACTATGGTACCACCTGAGGGCCCCCGTCCTTATAGATGAATGCCAACCGTGTATATACCACACGATTGCAACGGCGCGCTTAAGCTCCAACTACTTAGTCTCTATAGAGGAGATGCTAGTGTCTAGTGTGGAGACTGCAGCCGGATCGATGGGGGCGATTGGTTGATGTCTACTGGCGCTCAAGTCGATCAGGCCTTTGGGGAGCACACAGATGTATGGTCCGGTTGATCAGAGCCTCATAAGACCCCGAATGTATTGTCCCGCATACTGCCTTTCGCCGTTCGAAAAGAACTGCGATTAGTGTTTATCGTAGCGGGTGCCGGAGGGCCTTAATTGGTTCCAGACGCTTTTTCTCAGTGATATAGAAGTTTAAGCGAAGAGGTATGTGTTCCGGGTCGTATACGGTCGGGGTTAAGAGCTTGGCACCATTTATAGCTAGCCAGTTTACGGATGCAGTCGGGCCGCGCTTATTGATACTTGCAAAACCGATCACTCGCTAATCATGTGGATAACCCATTATTGATCGGCCTATTTCTTTCAATGGGATCTGGTGGAGGATCCCGGTAAATGCAGCCAGTCCCACTGCCTCGAGCTATTCTGGTCTGGGACCATCAATAAATTCTGACTTGTCCGTGTAAGCCACCTCGTACAGTGATTCCGAGGACACCACTCAGCGTCGCAGTTAGTATTTCTCGGTGTATGACCTAGCTGTTCCTTGTCGCAACTACCAATGAGCCCCACAACCGGAGTCTTTCCCAGCTAATTCGAGCTCAACTACGCTCGAAAAGGATCACCCATTTACTGTTATACGTGCAGGAGCTACAGTGCCAATACATGCAGTAAAGAGTTTGGGGTTACGTGGGACCTCTGTCCCTCACCTAACTTTTTGGGTACTTGAGCCGCGGGGCTTGTCACGGACACCTAGGCCGAGCTCTCAATCCCAGGAGAAACAGG

In [57]:
np.array(matcher.a.split)

array('GAGGATCCCGGTAAATGCAGCCATTCCCACTGCCTCGAGCTATTCTGGTCTGGGACCATCAATAAATTCTGTCTTGTCCGTGTAAGCCACCTCGTACAGT',
      dtype='<U100')

In [8]:
# Python code for a dynamic programming solution to the longest common substring
# taken from Geeks for Geeks
# https://www.geeksforgeeks.org/longest-common-substring-dp-29/

from functools import lru_cache
from operator import itemgetter
 
def longest_common_substring(x: str, y: str) -> (int, int, int):
     
    # function to find the longest common substring
 
    # Memorizing with maximum size of the memory as 1
    @lru_cache(maxsize=1) 
     
    # function to find the longest common prefix
    def longest_common_prefix(i: int, j: int) -> int:
       
        if 0 <= i < len(x) and 0 <= j < len(y) and x[i] == y[j]:
            return 1 + longest_common_prefix(i + 1, j + 1)
        else:
            return 0
 
    # digonally computing the subproplems
    # to decrease memory dependency
    def digonal_computation():
         
        # upper right trianle of the 2D array
        for k in range(len(x)):       
            yield from ((longest_common_prefix(i, j), i, j)
                        for i, j in zip(range(k, -1, -1),
                                    range(len(y) - 1, -1, -1)))
         
        # lower left triangle of the 2D array
        for k in range(len(y)):       
            yield from ((longest_common_prefix(i, j), i, j)
                        for i, j in zip(range(k, -1, -1),
                                    range(len(x) - 1, -1, -1)))
 
    # returning the maximum of all the subproblems
    return max(digonal_computation(), key=itemgetter(0), default=(0, 0, 0))


longest_common_substring("Hello", "jello")

(4, 1, 1)